In [ ]:
import numpy as np
import sympy
import cirq
import tensorflow as tf
import tensorflow_quantum as tfq
import matplotlib.pyplot as plt
from tqdm import tqdm
import matplotlib.pyplot as plt
import gc

n_qubits = 4

qubits = cirq.GridQubit.rect(1, n_qubits)
g=1
J=1.21
observable = [-float(0.5*g)*cirq.Z.on(q) for q in qubits] 
for q in range(len(qubits)):
    observable.append(-float(0.5*J)*cirq.X.on(qubits[q])*cirq.X.on(qubits[(q+1)%len(qubits)]))



gate=[cirq.rz, cirq.rx, cirq.rz]
for k in range(25):
    circuit = []
    symbols=[]
    for ii in range(3):
        for k in range(len(qubits)):
            for j in range(3):
                symbol="th_"+str(ii)+"_"+str(k)+"_"+str(j)
                symbols.append(symbol)
                circuit.append(gate[j](sympy.Symbol(symbol)).on(qubits[k]))
    #for k in range(len(qubits)):
    #    circuit.append(cirq.CNOT(qubits[k], qubits[(k+1)%len(qubits)]))
    circuit = cirq.Circuit(circuit)

    circuit_input = tf.keras.Input(shape=(), dtype=tf.string)
    output = tfq.layers.Expectation(backend=cirq.DensityMatrixSimulator(noise=cirq.depolarize(0.00001)))(
    circuit_input,
    symbol_names=symbols,
    operators=tfq.convert_to_tensor([observable]),
    initializer=tf.keras.initializers.RandomNormal()) #we may change this!!!

    model = tf.keras.Model(inputs=circuit_input, outputs=output)
    adam = tf.keras.optimizers.Adam(learning_rate=0.01)
    model.compile(optimizer=adam, loss='mse')

    tfqcircuit = tfq.convert_to_tensor([circuit])
    qoutput = tf.ones((1, 1))*-2.*n_qubits
    
    model.set_weights([tf.random.uniform((len(symbols),))])
    model.fit(x=tfqcircuit, y=qoutput,validation_steps=None,
              steps_per_epoch=1, batch_size=1, epochs=20, verbose=1, 
              callbacks=[tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)])
    del model
    del output
    del circuit
    gc.collect()

Train on 1 samples
Epoch 1/20
1/1 [==============================] - 10s 10s/sample - loss: 59.1337
Epoch 2/20
1/1 [==============================] - 8s 8s/sample - loss: 59.0477
Epoch 3/20
1/1 [==============================] - 8s 8s/sample - loss: 58.9735
Epoch 4/20
1/1 [==============================] - 8s 8s/sample - loss: 58.9107
Epoch 5/20
1/1 [==============================] - 8s 8s/sample - loss: 58.8564
Epoch 6/20
1/1 [==============================] - 8s 8s/sample - loss: 58.8080
Epoch 7/20
1/1 [==============================] - 8s 8s/sample - loss: 58.7637
Epoch 8/20
1/1 [==============================] - 8s 8s/sample - loss: 58.7235
Epoch 9/20
1/1 [==============================] - 8s 8s/sample - loss: 58.6871
Epoch 10/20
1/1 [==============================] - 8s 8s/sample - loss: 58.6547
Epoch 11/20
1/1 [==============================] - 9s 9s/sample - loss: 58.6262
Epoch 12/20
1/1 [==============================] - 9s 9s/sample - loss: 58.6012
Epoch 13/20
1/1 [===========

Process ForkPoolWorker-214:
Process ForkPoolWorker-213:
Process ForkPoolWorker-215:
Process ForkPoolWorker-216:
Traceback (most recent call last):
  File "/home/cooper-cooper/.local/lib/python3.6/site-packages/multiprocess/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/cooper-cooper/.local/lib/python3.6/site-packages/multiprocess/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/cooper-cooper/.local/lib/python3.6/site-packages/multiprocess/process.py", line 258, in _bootstrap
    self.run()
  File "/home/cooper-cooper/.local/lib/python3.6/site-packages/multiprocess/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
Traceback (most recent call last):
  File "/home/cooper-cooper/.local/lib/python3.6/site-packages/multiprocess/pool.py", line 47, in starmapstar
    return list(itertools.starmap(args[0], args[1]))
  File "/home/cooper-cooper/.lo

  File "/home/cooper-cooper/.local/lib/python3.6/site-packages/cirq/circuits/circuit.py", line 1803, in _resolve_operations
    op, param_resolver))
  File "/home/cooper-cooper/.local/lib/python3.6/site-packages/cirq/circuits/circuit.py", line 1694, in _resolve_parameters_
    param_resolver)
  File "/home/cooper-cooper/.local/lib/python3.6/site-packages/cirq/ops/eigen_gate.py", line 348, in _resolve_parameters_
    exponent=param_resolver.value_of(self._exponent))
  File "/home/cooper-cooper/.local/lib/python3.6/site-packages/cirq/circuits/circuit.py", line 1803, in _resolve_operations
    op, param_resolver))
  File "/home/cooper-cooper/.local/lib/python3.6/site-packages/cirq/study/resolver.py", line 121, in value_of
    v = value.subs(self.param_dict)
  File "/home/cooper-cooper/.local/lib/python3.6/site-packages/cirq/ops/gate_operation.py", line 151, in _resolve_parameters_
    resolved_gate = protocols.resolve_parameters(self.gate, resolver)
  File "/home/cooper-cooper/.local/lib/